# Import Libraries

In [1]:
import pyspark
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from dotenv import dotenv_values

# Start Pyspark Session and Load Environment Variables

In [2]:
config = dotenv_values('.env')
spark=SparkSession.builder.appName('EDA').getOrCreate()
spark

In [4]:
train_labels = spark.read.csv(config["TRAIN_LABELS_PATH"], header=True, inferSchema=True)

In [5]:
train_labels.head(5)

[Row(customer_ID='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a', target=0),
 Row(customer_ID='00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5', target=0),
 Row(customer_ID='00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1', target=0),
 Row(customer_ID='000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc', target=0),
 Row(customer_ID='00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed', target=0)]

# Count Null Values in PySpark Dataframe

In [6]:
from pyspark.sql.functions import isnan, when, count, col

In [7]:
train_labels.select([count(when(isnan(c),c)).alias(c) for c in train_labels.columns]).show()

+-----------+------+
|customer_ID|target|
+-----------+------+
|          0|     0|
+-----------+------+



# Visualize the Discrepancy Between True and False Target Values